In [ ]:
import numpy as np
import os
import shutil
import random
import torch
import sklearn.utils

from vae.model import VAE
# from gen_model.model import AutoDropoutNN
from loader.dataloader import MIDI_Loader,MIDI_Render
from loader.chordloader import Chord_Loader
from torch.nn import functional as F

pitch_num = 130
chord_num = 12
rest_pitch = 129
hold_pitch = 128
none_chord = 24
recog_level = "Mm"
min_step = 0.125 # for vae
total_len = 256 # 32.0 s
known_len = 128 # 16.0 s
shift_len = 32 # 4.0 s
vae_len = 32 # 4.0s


def vae_make_one_hot_data(train_data):
    print("convert data to one-hot...",flush = True)
    train_size = len(train_data)

    train_x = np.zeros((train_size,total_len,pitch_num), dtype = np.int32)
    train_cond = np.zeros((train_size,total_len,chord_num), dtype = np.int32)
    # train_gd = np.zeros((train_size,total_len), dtype = np.int32)
    train_x = []
    train_cond = []
    cl = Chord_Loader()
    # process with bi-directional issue

    for i,data in enumerate(train_data):
        if i >= train_size:
            break
        mi = data["notes"]
        ci = data["chords"]
        prev = rest_pitch
        temp_x = np.zeros((len(data["notes"]),pitch_num), dtype = np.int32)
        temp_cond = np.zeros((len(data["notes"]),chord_num), dtype = np.int32)
        for j,value in enumerate(mi):
            temp_x[j, value] = 1
#         print(ci)
        for j,value in enumerate(ci):
            cname = cl.index2name(value)
            cnotes = cl.name2note(cname)
            # print(cnotes)
#             print(cname)
            if cnotes is None:
#                 print("??")
                continue
            for k in cnotes:
                temp_cond[j,k % 12] = 1
#         print(temp_cond)
        train_x.append(temp_x)
        train_cond.append(temp_cond)
    print("convert success！",flush = True)
    train_x = np.array(train_x)
    train_cond = np.array(train_cond)
    return [train_x,train_cond]

test_data = np.load("train_data_8_crop.npy",allow_pickle=True)


# train_x,train_cond = vae_make_one_hot_data(train_data)
test_x,test_cond = vae_make_one_hot_data(test_data)
# validate_x,validate_cond = vae_make_one_hot_data(validate_data)

print(test_x.shape)
print(test_cond.shape)
# for i in range(len(test_x)):
#     while len(test_x[i]) % vae_len != 0:
#         test_x[i] = np.delete(test_x[i],-1,0)
#         test_cond[i] = np.delete(test_cond[i],-1,0)

convert data to one-hot...


In [12]:
# [np.argmax(d) for d in test_x[0]]

[64,
 66,
 128,
 68,
 128,
 69,
 128,
 128,
 68,
 69,
 128,
 71,
 128,
 73,
 128,
 75,
 128,
 76,
 128,
 73,
 128,
 71,
 128,
 73,
 128,
 71,
 128,
 68,
 128,
 73,
 128,
 75,
 128,
 76,
 128,
 73,
 128,
 71,
 128,
 68,
 128,
 66,
 128,
 68,
 128,
 64,
 128,
 64,
 64,
 64,
 128,
 128,
 128,
 71,
 128,
 128,
 76,
 71,
 128,
 69,
 128,
 66,
 128,
 128,
 64,
 66,
 128,
 68,
 128,
 69,
 128,
 128,
 68,
 69,
 128,
 71,
 128,
 73,
 128,
 75,
 128,
 76,
 128,
 73,
 128,
 71,
 128,
 73,
 128,
 71,
 128,
 68,
 128,
 73,
 128,
 75,
 128,
 76,
 128,
 73,
 128,
 71,
 128,
 68,
 128,
 66,
 128,
 68,
 128,
 64,
 128,
 64,
 64,
 64,
 128,
 128,
 128,
 68,
 128,
 71,
 128,
 64,
 128,
 71,
 128,
 66,
 128,
 71,
 128,
 70,
 128,
 71,
 128,
 68,
 128,
 71,
 128,
 64,
 128,
 71,
 128,
 66,
 128,
 59,
 128,
 59,
 128,
 128,
 128,
 68,
 128,
 71,
 128,
 64,
 128,
 71,
 128,
 66,
 128,
 70,
 128,
 73,
 128,
 76,
 128,
 75,
 128,
 73,
 128,
 71,
 128,
 70,
 128,
 71,
 128,
 71,
 71,
 71,
 128,
 128,
 128,
 71,

In [13]:
#initialize model
model = VAE(130, 2048, 3, 12, 128, 128, 32)
model.eval()
dic = torch.load("vae/tr_chord.pt")
for name in list(dic.keys()):
    dic[name.replace('module.', '')] = dic.pop(name)
model.load_state_dict(dic)
if torch.cuda.is_available():
    model = model.cuda()
print(model)

vae_out = []
for sindex in range(len(test_x)):
    a = test_x[sindex]
    b = test_cond[sindex]
#     print(a.shape)
#     print(b.shape)
    a = np.reshape(a, (-1, 32, 130))
    b = np.reshape(b, (-1, 32, 12))
#     print(a.shape)
#     print(b.shape)
    a = torch.from_numpy(a).float()
    b = torch.from_numpy(b).float()
    if torch.cuda.is_available():
        a = a.cuda()
        b = b.cuda()
    res = model.encoder(a,b)
    z1 = res[0].loc.detach().cpu()#     z1 = torch.from_numpy(z1).float()
#     z2 = torch.from_numpy(z2).float()
#     z1 = z1.cuda()
#     z2 = z2.cuda()
#     res = model.decoder(z1,z2,b)
#     print(res.detach().cpu().numpy().shape)
#     q = np.asarray((res.detach().cpu().numpy()))
#     temp = []
#     for i in q:
#         for j in i:
#             temp.append(np.argmax(j))
#     q = np.array(temp)
#     print(q)
#     song = {"notes":q, "chords": test_data[sindex]["chords"]}
    
#     render = MIDI_Render(datasetName = "Nottingham", minStep= min_step)
#     render.data2midi(data = song,output = "test.mid")
#     render.data2midi(data = test_data[sindex],output = "real.mid")).numpy()
    z2 = res[1].loc.detach().cpu().numpy()
    vae_out.append([z1,z2,b.detach().cpu().numpy()])
    print("finish %d" % (sindex))
#     print(z1)
#     print(z1.shape)
#     print(z2)
#     print(z2.shape)

np.save("vae_train_data.npy", np.array(vae_out))

VAE(
  (gru_0): GRU(142, 2048, batch_first=True, bidirectional=True)
  (linear_mu): Linear(in_features=4096, out_features=256, bias=True)
  (linear_var): Linear(in_features=4096, out_features=256, bias=True)
  (grucell_0): GRUCell(131, 2048)
  (grucell_1): GRUCell(273, 2048)
  (grucell_2): GRUCell(2048, 2048)
  (linear_init_0): Linear(in_features=128, out_features=2048, bias=True)
  (linear_out_0): Linear(in_features=2048, out_features=3, bias=True)
  (linear_init_1): Linear(in_features=128, out_features=2048, bias=True)
  (linear_out_1): Linear(in_features=2048, out_features=130, bias=True)
)
finish 0
finish 1
finish 2
finish 3
finish 4
finish 5
finish 6
finish 7
finish 8
finish 9
finish 10
finish 11
finish 12
finish 13
finish 14
finish 15
finish 16
finish 17
finish 18
finish 19
finish 20
finish 21
finish 22
finish 23
finish 24
finish 25
finish 26
finish 27
finish 28
finish 29
finish 30
finish 31
finish 32
finish 33
finish 34
finish 35
finish 36
finish 37
finish 38
finish 39
finish 40

finish 702
finish 703
finish 704
finish 705
finish 706
finish 707
finish 708
finish 709
finish 710
finish 711
finish 712
finish 713
finish 714
finish 715
finish 716
finish 717
finish 718
finish 719
finish 720
finish 721
finish 722
finish 723
finish 724
finish 725
finish 726
finish 727
finish 728
finish 729
finish 730
finish 731
finish 732
finish 733
finish 734
finish 735
finish 736
finish 737
finish 738
finish 739
finish 740
finish 741
finish 742
finish 743
finish 744
finish 745
finish 746
finish 747
finish 748
finish 749
finish 750
finish 751
finish 752
finish 753
finish 754
finish 755
finish 756
finish 757
finish 758
finish 759
finish 760
finish 761
finish 762
finish 763
finish 764
finish 765
finish 766
finish 767
finish 768
finish 769
finish 770
finish 771
finish 772
finish 773
finish 774
finish 775
finish 776
finish 777
finish 778
finish 779
finish 780
finish 781
finish 782
finish 783
finish 784
finish 785
finish 786
finish 787
finish 788
finish 789
finish 790
finish 791
finish 792

finish 1410
finish 1411
finish 1412
finish 1413
finish 1414
finish 1415
finish 1416
finish 1417
finish 1418
finish 1419
finish 1420
finish 1421
finish 1422
finish 1423
finish 1424
finish 1425
finish 1426
finish 1427
finish 1428
finish 1429
finish 1430
finish 1431
finish 1432
finish 1433
finish 1434
finish 1435
finish 1436
finish 1437
finish 1438
finish 1439
finish 1440
finish 1441
finish 1442
finish 1443
finish 1444
finish 1445
finish 1446
finish 1447
finish 1448
finish 1449
finish 1450
finish 1451
finish 1452
finish 1453
finish 1454
finish 1455
finish 1456
finish 1457
finish 1458
finish 1459
finish 1460
finish 1461
finish 1462
finish 1463
finish 1464
finish 1465
finish 1466
finish 1467
finish 1468
finish 1469
finish 1470
finish 1471
finish 1472
finish 1473
finish 1474
finish 1475
finish 1476
finish 1477
finish 1478
finish 1479
finish 1480
finish 1481
finish 1482
finish 1483
finish 1484
finish 1485
finish 1486
finish 1487
finish 1488
finish 1489
finish 1490
finish 1491
finish 1492
fini

finish 2093
finish 2094
finish 2095
finish 2096
finish 2097
finish 2098
finish 2099
finish 2100
finish 2101
finish 2102
finish 2103
finish 2104
finish 2105
finish 2106
finish 2107
finish 2108
finish 2109
finish 2110
finish 2111
finish 2112
finish 2113
finish 2114
finish 2115
finish 2116
finish 2117
finish 2118
finish 2119
finish 2120
finish 2121
finish 2122
finish 2123
finish 2124
finish 2125
finish 2126
finish 2127
finish 2128
finish 2129
finish 2130
finish 2131
finish 2132
finish 2133
finish 2134
finish 2135
finish 2136
finish 2137
finish 2138
finish 2139
finish 2140
finish 2141
finish 2142
finish 2143
finish 2144
finish 2145
finish 2146
finish 2147
finish 2148
finish 2149
finish 2150
finish 2151
finish 2152
finish 2153
finish 2154
finish 2155
finish 2156
finish 2157
finish 2158
finish 2159
finish 2160
finish 2161
finish 2162
finish 2163
finish 2164
finish 2165
finish 2166
finish 2167
finish 2168
finish 2169
finish 2170
finish 2171
finish 2172
finish 2173
finish 2174
finish 2175
fini

finish 2778
finish 2779
finish 2780
finish 2781
finish 2782
finish 2783
finish 2784
finish 2785
finish 2786
finish 2787
finish 2788
finish 2789
finish 2790
finish 2791
finish 2792
finish 2793
finish 2794
finish 2795
finish 2796
finish 2797
finish 2798
finish 2799
finish 2800
finish 2801
finish 2802
finish 2803
finish 2804
finish 2805
finish 2806
finish 2807
finish 2808
finish 2809
finish 2810
finish 2811
finish 2812
finish 2813
finish 2814
finish 2815
finish 2816
finish 2817
finish 2818
finish 2819
finish 2820
finish 2821
finish 2822
finish 2823
finish 2824
finish 2825
finish 2826
finish 2827
finish 2828
finish 2829
finish 2830
finish 2831
finish 2832
finish 2833
finish 2834
finish 2835
finish 2836
finish 2837
finish 2838
finish 2839
finish 2840
finish 2841
finish 2842
finish 2843
finish 2844
finish 2845
finish 2846
finish 2847
finish 2848
finish 2849
finish 2850
finish 2851
finish 2852
finish 2853
finish 2854
finish 2855
finish 2856
finish 2857
finish 2858
finish 2859
finish 2860
fini

finish 3461
finish 3462
finish 3463
finish 3464
finish 3465
finish 3466
finish 3467
finish 3468
finish 3469
finish 3470
finish 3471
finish 3472
finish 3473
finish 3474
finish 3475
finish 3476
finish 3477
finish 3478
finish 3479
finish 3480
finish 3481
finish 3482
finish 3483
finish 3484
finish 3485
finish 3486
finish 3487
finish 3488
finish 3489
finish 3490
finish 3491
finish 3492
finish 3493
finish 3494
finish 3495
finish 3496
finish 3497
finish 3498
finish 3499
finish 3500
finish 3501
finish 3502
finish 3503
finish 3504
finish 3505
finish 3506
finish 3507
finish 3508
finish 3509
finish 3510
finish 3511
finish 3512
finish 3513
finish 3514
finish 3515
finish 3516
finish 3517
finish 3518
finish 3519
finish 3520
finish 3521
finish 3522
finish 3523
finish 3524
finish 3525
finish 3526
finish 3527
finish 3528
finish 3529
finish 3530
finish 3531
finish 3532
finish 3533
finish 3534
finish 3535
finish 3536
finish 3537
finish 3538
finish 3539
finish 3540
finish 3541
finish 3542
finish 3543
fini